## Selecting Features

In [8]:
import pandas as pd
import numpy as np

from  sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.inspection import permutation_importance

In [2]:
all_data_df = pd.read_csv('all_data_df.csv')
all_data_df.index = all_data_df['SEQN']

mental_health_df = all_data_df.loc[:, 'DPQ010':'DPQ100'].dropna(how='all')
all_data_df = all_data_df.loc[mental_health_df.index]

def mh(x):
    if x == '\.':
        return 'missing'
    elif x == 1:
        return 'several days'
    elif x == 2:
        return 'more than half the days'
    elif x == 3:
        return 'nearly every day'
    elif x == 7:
        return 'refused'
    elif x == 9:
        return "don't know"
    else:
        return 'not at all'

for col in mental_health_df.columns:
    mental_health_df[col] = mental_health_df[col].apply(lambda x: mh(x))

def calc(row):
    sum = 0
    for i in ['DPQ010', 'DPQ020', 'DPQ030', 'DPQ040', 
              'DPQ050', 'DPQ060', 'DPQ070','DPQ080', 
              'DPQ090', 'DPQ100']:
        if row[i] == 'several days':
            sum += 1
        if row[i] == 'more than half the days':
            sum += 2
        if row[i] == 'nearly every day':
            sum += 3
    return sum

mental_health_df['labels_raw'] = mental_health_df.apply(calc, axis=1)
mental_health_df['labels'] = mental_health_df['labels_raw'].apply(lambda x: 1 if x >= 10 else 0)

/Users/flowerbed/Desktop/School/9th Grade 2020 - 2021/2021 UCSF AI4ALL/GitHub/ai4all_nhanes/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (600,601) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
features = [
    #diabetes
    'DID040','DIQ160','DIQ170','DIQ172','DIQ175A','DIQ175B','DIQ175C',
    'DIQ175D','DIQ175E','DIQ175F','DIQ175G','DIQ175H','DIQ175I','DIQ175J','DIQ175K',
    'DIQ175L', 'DIQ175M','DIQ175N','DIQ175O','DIQ175P','DIQ175Q','DIQ175R','DIQ175S',
    'DIQ175T','DIQ175U','DIQ175V','DIQ175W','DIQ180','DIQ050','DID060','DIQ060U',
    'DIQ070','DIQ230','DIQ240','DID250','DID260','DIQ260U','DIQ275','DIQ280','DIQ291',
    'DIQ300S','DIQ300D','DID310S','DID310D','DID320','DID330','DID341','DID350',
    'DIQ350U','DIQ360','DIQ080', 
    #sleep disorder
    'SLQ050', 
    #physical activity
    'PAQ605','PAQ610','PAD615','PAQ620','PAQ625','PAD630','PAQ635','PAQ640','PAD645',
    'PAQ650','PAQ655','PAD660','PAQ665','PAQ670','PAD675','PAD680','PAQ706','PAQ710',
    'PAQ715', 
    #weight history
    'WHD010','WHD020','WHQ030','WHQ040','WHD050','WHQ060','WHQ070','WHD080A','WHD080B',
    'WHD080C','WHD080D','WHD080E','WHD080F','WHD080G','WHD080H','WHD080I','WHD080J',
    'WHD080K','WHD080M','WHD080N','WHD080O','WHD080P','WHD080Q','WHD080R','WHD080S',
    'WHD080T','WHD080L','WHD110','WHD120','WHD130','WHD140','WHQ150', 
    #early childhood
    'ECD010','ECQ020','ECD070A','ECD070B','ECQ080','ECQ090','WHQ030E','MCQ080E',
    'ECQ150', 
    #alcohol issues
    'ALQ101','ALQ110','ALQ120Q','ALQ120U','ALQ130','ALQ141Q','ALQ141U','ALQ151',
    #early childhood
    'ECD010','ECQ020','ECD070A','ECD070B','ECQ080','ECQ090','WHQ030E','MCQ080E','ECQ150', 
    #hospital access
    'HUQ010','HUQ020','HUQ030','HUQ071','HUD080','HUQ090', 
    #health status
    'HSD010','HSQ500','HSQ510','HSQ520','HSQ571','HSQ580','HSQ590','HSAQUEX',  
    #income
    'INQ012','INQ030','INQ060','INQ080','INQ090','INQ132','INQ140','INQ150',
    'IND235','INDFMMPI','INDFMMPC', 
    #housing
    'HOD050','HOQ065', 
    #occupation
    'OCD150','OCQ180','OCQ210','OCQ260','OCD270','OCQ380','OCD390G','OCD395', 
    #diet nutrition
    'DBQ010','DBD030','DBD041','DBD050','DBD055','DBD061','DBQ073A','DBQ073B','DBQ073C',
    'DBQ073D','DBQ073E','DBQ073U','DBQ700','DBQ197','DBQ223A','DBQ223B','DBQ223C',
    'DBQ223D','DBQ223E','DBQ223U','DBQ229','DBQ235A','DBQ235B','DBQ235C','DBQ301',
    'DBQ330','DBQ360','DBQ370','DBD381','DBQ390','DBQ400','DBD411','DBQ421','DBQ424',
    'DBD895','DBD900','DBD905','DBD910', 
    #drug use
    'DUQ200','DUQ210','DUQ211','DUQ213','DUQ215Q','DUQ215U','DUQ217','DUQ219','DUQ220Q',
    'DUQ220U','DUQ230','DUQ240','DUQ250','DUQ260','DUQ270Q','DUQ270U','DUQ272','DUQ280',
    'DUQ290','DUQ300','DUQ310Q','DUQ310U','DUQ320','DUQ330','DUQ340','DUQ350Q','DUQ350U',
    'DUQ352','DUQ360','DUQ370','DUQ380A','DUQ380B','DUQ380C','DUQ380D','DUQ380E',
    'DUQ390','DUQ400Q','DUQ400U','DUQ410','DUQ420', 
    
]

In [83]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

X = all_data_df[features]
y = mental_health_df['labels']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

rand_for_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('enc', OneHotEncoder(sparse=False, handle_unknown='ignore')), 
    ('red', PCA(n_components=10)),
    ('clf', RandomForestClassifier(max_depth=7, n_estimators=1000)),
])
rand_for_pipe.fit(X_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('enc', OneHotEncoder(handle_unknown='ignore', sparse=False)),
                ('red', PCA(n_components=10)),
                ('clf',
                 RandomForestClassifier(max_depth=7, n_estimators=1000))])

In [84]:
r = permutation_importance(
    rand_for_pipe, 
    X_val, 
    y_val,
    n_repeats=10,
    n_jobs=-1,
    random_state=42
)

In [85]:
feature_importances = pd.DataFrame.from_dict(
    {
        'importance_means': r['importances_mean'],
        'importances_std': r['importances_std']
    }, orient='columns'
)
feature_importances.index = X_val.columns

In [86]:
feature_importances.sort_values('importance_means', ascending=False)
feature_importances.sort_values('importance_means', ascending=False).iloc[-10:]

,importance_means,importances_std
WHD080N,0.0,0.0
WHD080M,0.0,0.0
WHD080K,0.0,0.0
WHD080J,0.0,0.0
WHD080I,0.0,0.0
WHD080H,0.0,0.0
WHD080G,0.0,0.0
WHD080F,0.0,0.0
WHD080E,0.0,0.0
DUQ400U,0.0,0.0


In [87]:
features = [
    #diabetes
    'DIQ170','DIQ175A','DIQ175B','DIQ175C',
    'DIQ175D','DIQ175E','DIQ175F','DIQ175G','DIQ175H','DIQ175I','DIQ175J','DIQ175K',
    'DIQ175L', 'DIQ175M','DIQ175N','DIQ175O','DIQ175P','DIQ175Q','DIQ175R','DIQ175S',
    'DIQ175T','DIQ175U','DIQ175V','DIQ175W','DIQ050','DIQ060U',
    'DIQ260U',
    #sleep disorder
    #physical activity
    'PAD680','PAQ706',
    #weight history
    'WHD080A','WHD080B',
    'WHD080C','WHD080D','WHD080E','WHD080F','WHD080G','WHD080H','WHD080I','WHD080J',
    'WHD080K','WHD080M','WHD080N','WHD080O','WHD080P','WHD080Q','WHD080R','WHD080S',
    'WHD080T','WHD080L',
    #early childhood
    'ECD010','ECQ020','ECD070A','ECD070B','ECQ080','ECQ090','WHQ030E','MCQ080E',
    'ECQ150', 
    #alcohol issues
    #early childhood
    'ECD010','ECQ020','ECD070A','ECD070B','ECQ080','ECQ090','WHQ030E','MCQ080E','ECQ150', 
    #hospital access
    'HUQ071','HUQ090', 
    #health status
    'HSQ520','HSAQUEX',  
    #income
    'INQ150',
    #housing
    #occupation
    #diet nutrition
    'DBQ010','DBD030','DBD041','DBD050','DBD055','DBD061','DBQ073A','DBQ073B','DBQ073C',
    'DBQ073D','DBQ073E','DBQ073U','DBQ700','DBQ223A','DBQ223B','DBQ223C',
    'DBQ223D','DBQ223E','DBQ223U',
    'DBQ330','DBD381','DBQ390','DBQ400','DBD411','DBQ421','DBQ424',
    #drug use
    'DUQ250','DUQ280',
    'DUQ290','DUQ300','DUQ310Q','DUQ310U','DUQ320',
    'DUQ360','DUQ380A','DUQ380B','DUQ380C','DUQ380D','DUQ380E',
    'DUQ400U'
]

- ran model 10 times and removed all features with negative importance_means values each time
- round 2: housing eliminated, sleep disorder eliminated, occupation eliminated
- round 4: alcohol issues eliminated
- round 8: no negative scores
- round 9: confirmation of no negative scores
- round 10: double-check to make sure there are no negative scores